# Run FoldX Stability Analysis

In [1]:
import pandas as pd
import subprocess
import shutil

In [3]:
## Load the experiments data
experiments = pd.read_excel('../../../tests/test_cases.xlsx', sheet_name='generated_seqs')
experiments.head()

,antigen_id,model,seq_id,h_chain,l_chain
0,BHRF1,peleke-phi-4,peleke-phi-4_BHRF1_01,QVQLVQSGAEVKKPGSSVKVSCKASGGTFNTGISWVRQAPGQGLEW...,DIVLTQSPATLSVTPGDSVSLSCRASQTISKNNLHWYQQKSHESPR...
1,BHRF1,peleke-phi-4,peleke-phi-4_BHRF1_02,EVQLEESGGGLVQPGGSLRLSCAASGFNVVDFSLHWVRQAPGKGLE...,DIQMTQTTSSLSASLGDRVTISCRASQDISNYLNWYQQKPDGTVKL...
2,BHRF1,peleke-phi-4,peleke-phi-4_BHRF1_03,QVQLQESGPGLVKPSQTLSLTCTVSGGSISSSYWSWIRQPPGKGLE...,DIVMTQSPLSLPVTLGQPASISCRSSQNGNTYLHWYQQKPGQSPRL...
3,BHRF1,peleke-phi-4,peleke-phi-4_BHRF1_04,SVEESGGRLVTPGTPLTLTCTVSGFSLSTSGVHWIRQAPGKGLEWL...,QAVLTQSSSASFSLGQSVTISCTGTSNIGAGYTYVSWYQQHPGKAP...
4,BHRF1,peleke-phi-4,peleke-phi-4_BHRF1_05,EVQLVESGGGLVQPGGSLRLSCAASGFTFSDTYIHWVRQAPGKGLE...,DIQMTQSPASLSASVGDRVTITCRASQSVSSAVAWYQQKPGKAPKL...


In [4]:
## Make blank dataframe for FoldX results
foldx_cols = [
    'seq_id',
    'Pdb', 'Total Energy', 'Backbone Hbond', 
    'Sidechain Hbond', 'Van der Waals', 'Electrostatics',
    'Solvation Polar', 'Solvation Hydrophobic', 'Van der Waals clashes',
    'Entropy Side Chain', 'Entropy Main Chain', 'Sloop Entropy',
    'Mloop Entropy', 'Cis Bond', 'Torsional Clash', 'Backbone Clash',
    'Helix Dipole', 'Water Bridge', 'Disulfide', 'Electrostatic Kon',
    'Partial Covalent Bonds', 'Energy Ionisation',
    'Entropy Complex', 'Residue Number'
]

foldx_df = pd.DataFrame(columns=foldx_cols)

In [ ]:
## Loop through each experiment
for index, row in experiments.iterrows():
    seq_id = row['seq_id']
    # pdb_name = row['best_model_name'].strip()
    pdb_name = 'antibody.pdb'

    ## Copy the best model PDB file to the current directory
    shutil.copyfile(f'../../../tests/structures/predicted_complexes/predicted__{seq_id}__esm3.pdb', f'./{pdb_name}')

    ## Construct the command to run the script
    command = [
        './executables/foldx5_1Mac/foldx_20251231',
        '--command=Stability',
        f'--pdb={pdb_name}'
    ]
    
    ## Execute the command
    subprocess.run(command, check=True)

    ## Read the output file and append to the DataFrame
    output_file = f'{pdb_name.replace('.pdb', '')}_0_ST.fxout'

    row_df = pd.read_csv(output_file, sep='\t', header=None, names=foldx_cols)
    row_df['seq_id'] = seq_id
    foldx_df = pd.concat([foldx_df, row_df], ignore_index=True)

## Save the results to a CSV file
foldx_df.to_csv('foldx_results.csv', index=False)


In [ ]:
foldx_df

In [ ]:
## Clean up the output files
subprocess.run(['rm', '*.fxout'], check=True)
subprocess.run(['rm', '*.pdb'], check=True)